# <div align='center'> 分类任务测试 </div>

In [9]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [17]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()



In [10]:
from k12libs.utils.nb_easy import k12ai_run_project

In [2]:
backbones = [
    'resnet18',           # 0
    'resnet50',           # 1
    'densenet121',        # 2
    'mobilenet_v2',       # 3
    'squeezenet1_0',      # 4
    'squeezenet1_1',      # 5
    'shufflenet_v2_x0_5', # 6
    'shufflenet_v2_x1_0', # 7
]

batch_sizes = [8, 16, 32, 64]
input_sizes = [28, 32, 64, 128, 256]

## 任务

In [4]:
project = k12ai_run_project(
    debug=True, tb_port=9002,
    framework='k12cv', task='cls',
    network=backbones[0], dataset='rmnist')

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…

## 调用API

In [126]:
op = 'train.start'
appId = 'k12ai.popkey'
user = 'user01'
uuid = 'uuid01'
dataset_name = 'mnist01'
host = '116.85.5.40'
port = 8119
uri = 'k12ai/framework/execute'
popapi = f'http://{host}:{port}/k12ai/private/popmsg'
params = project.get_all_kv()
params['data.data_dir'] = f'oss://data-platform/datasets/{user}/{uuid}/{dataset_name}'
params['dataset'] = 'listdir'
params['data.dataset'] = dataset_name
params['solver.max_epoch'] = 160
params['solver.display_iter'] = 10
params['solver.test_interval'] = 20
params = json.dumps(params, indent=4)
print(params)

{
    "task": "cls",
    "method": "image_classifier",
    "data.include_val": false,
    "dataset": "listdir",
    "data.workers": 1,
    "data.drop_last": false,
    "data.image_tool": "cv2",
    "data.input_mode": "BGR",
    "solver.display_iter": 10,
    "solver.test_interval": 20,
    "solver.save_iters": 1024,
    "train.batch_size": 32,
    "val.batch_size": 32,
    "test.batch_size": 32,
    "train.data_transformer.fit_stride": 1,
    "train.data_transformer.size_mode": "fix_size",
    "train.data_transformer.input_size": [
        28,
        28
    ],
    "train.data_transformer.align_method": "scale_and_pad",
    "train.data_transformer.pad_mode": "pad_border",
    "_k12.train.random_border.bool": false,
    "_k12.train.random_brightness.bool": false,
    "_k12.train.random_contrast.bool": false,
    "_k12.train.random_crop.bool": false,
    "_k12.train.random_det_crop.bool": false,
    "_k12.train.random_gauss_blur.bool": false,
    "_k12.train.random_hsv.bool": false,
    

In [129]:
data = json.loads('''{
    "op":"%s",
    "token": "123456",
    "appId": "%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, appId, user, uuid, params))

api = 'http://%s:%d/%s' % (host, port, uri)
print("Input:\n")
print(data)
json.loads(requests.post(url=api, json=data).text)
def show_progress(popapi, msgkey, waitcnt=10):
    cnt = 1
    while True:
        result = json.loads(requests.get(url=f'{popapi}?key={msgkey}').text)
        for r in result:
            r = json.loads(r)
            for d in r['data']:
                if d['type'] == 'text':
                    p = d['data']
                    if 'progress' in p['title']:
                        v = int(float(p['payload']))
                        print_progress_bar(v)
                        if v >= 100:
                            return
            cnt = 0
        cnt += 1
        time.sleep(1)
        if cnt > waitcnt:
            print('timeout')
            break
print("\n\n")
show_progress(API_POPMSG, appId + '.metrics', 100)

Input:

{'op': 'train.start', 'token': '123456', 'appId': 'k12ai.popkey', 'user': 'user01', 'service_name': 'k12cv', 'service_uuid': 'uuid01', 'service_params': {'task': 'cls', 'method': 'image_classifier', 'data.include_val': False, 'dataset': 'listdir', 'data.workers': 1, 'data.drop_last': False, 'data.image_tool': 'cv2', 'data.input_mode': 'BGR', 'solver.display_iter': 10, 'solver.test_interval': 20, 'solver.save_iters': 1024, 'train.batch_size': 32, 'val.batch_size': 32, 'test.batch_size': 32, 'train.data_transformer.fit_stride': 1, 'train.data_transformer.size_mode': 'fix_size', 'train.data_transformer.input_size': [28, 28], 'train.data_transformer.align_method': 'scale_and_pad', 'train.data_transformer.pad_mode': 'pad_border', '_k12.train.random_border.bool': False, '_k12.train.random_brightness.bool': False, '_k12.train.random_contrast.bool': False, '_k12.train.random_crop.bool': False, '_k12.train.random_det_crop.bool': False, '_k12.train.random_gauss_blur.bool': False, '_k12.t

[metrics doc](http://116.85.5.40:8118/notebooks/docs/metrics_types.ipynb)

## 压测 (P40 跑10个任务)

In [ ]:
# from k12libs.utils.nb_easy import k12ai_train_execute
# keys = k12ai_train_execute(
#     framework='k12cv', task='cls',
#     network=backbones[-1], dataset='rmnist',
#     batchsize=32, inputsize=32, epoch_num=20, run_num=10)